## Quantum Approximate Optimisation Algorithm in Python

This notebook demostrates how the package in this repository can be used to solve quadratic unconstrained binary optimisation (QUBO) problems, expressed in the context of the max-cut problem from graph theory, using a quantum algorithm.

We start by first defining the graph using the $\texttt{networkx}$ package in Python. In this example we will be considering a quite simple complete graph with 4 vertices, where the weights of each edge are
uniformly distributed between 0 and 1.

In [1]:
import re
import qiskit
import json
import time
import numpy as np

from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.converters import circuit_to_dag

from src_code import build_operators
from src_code import graph_generator, get_data, build_operators

In [2]:
# Draw Graph
# pos=nx.circular_layout(graph)
# nx.draw_networkx(graph, pos)
# labels = nx.get_edge_attributes(graph,'weight')
# for edge in labels:
#     labels[edge] = round(labels[edge], 3)
# tmp = nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)


Once we have generated the graph, we can define the Ising Hamiltonian whose maximum energy eigenstate corresponds to the solution to the problem. This is given by $$\hat{H}_C=-\frac{1}{4}\sum_{i,j=0}^{n-1}W_{ij}\hat{Z}_i\hat{Z}_j.$$ It should be noted that the eigenvalues of this Hamiltonian are not exactly equal to the values of their corresponding cuts. They differ by a constant term $A = \frac{1}{4}\sum_{i,j=0}^{n-1}W_{ij}$, which is removed as it doesn't influence to the operation of the algorithm. It is later added back in to generate correct results. Additionally, we can define all the commutators $$\left[\hat{H}_C,-i\hat{A}\right]\quad\forall \quad\hat{A}\in\mathcal{P}.$$ These are used to find the gradients in parameter space at each iteration which are used to determine which mixer operator from the pool $\mathcal{P}$ to be added to the ADAPT-QAOA circuit in the current layer.

In [3]:
no_vertices = 3
graph = graph_generator.generate_graph(no_vertices)

weights = build_operators.cut_hamiltonian(graph[0])[1]

gradient_ops_dict = build_operators.build_all_mixers(graph[0])

filtered_gradient_ops_dict = {key: value for key, value in gradient_ops_dict.items() if 'Y' not in key}

In order to make the generation of the unitaries in the ansatz quicker between iterations of both the overall algorithm and the classical optimsation scheme, it is useful to pre-compute all the Pauli strings appearing in the exponents of the unitaries. This then allows one to use the identity $$e^{i\alpha\hat{P}}=\cos(\alpha)\hat{I}+i\sin(\alpha)\hat{P}$$ for single Pauli strings, to find the unitaries using simple floating point arithmetic, rather than matrix exponentiation during each iteration.

In [4]:
pauli_ops_dict = build_operators.build_all_paulis(no_vertices)
filtered_pauli_ops_dict = {key: value for key, value in pauli_ops_dict.items() if 'Y' not in key}


All this pre-computation allows for significant speed-up in the execution of the algorithms. We now move on to actually running QAOA on the graph. We first perform the standard non-adaptive algorithm. To do so we require to pick a specific depth for the circuit, i.e., the number of layers it will contain. We set this to 5.

In [22]:
circuit_depth = 2
qaoa_solution = get_data.run_standard_qaoa(graph[0], depth=circuit_depth, pauli_ops_dict=pauli_ops_dict)

In [23]:
for key in qaoa_solution:
    print(key+':', qaoa_solution[key])

cut_approx_ratio: 0.9975880629031183
ham_approx_ratio: 0.9947372465741234
optimised_Hamiltonian_unitary_parameters: [-1.6408562359564474, 3.9669387104283538]
optimised_mixer_unitary_parameters: [1.1498712384775749, 0.401598125972599]


We now move on the ADAPT-QAOA. We use the same maximum depth of 5 as above.

In [24]:
adapt_qaoa_solution = get_data.run_adapt_qaoa(graph[0], filtered_pauli_ops_dict, filtered_gradient_ops_dict, circuit_depth)

Initial Cut Approximation Ratio: 0.5416967314063463 

Finding Best Mixer for layer 1...
	Best mixer is standard_x with gradient magnitude 0.026390259272443036

Optimising layer 1...
	Initial Parameter Guesses: [0.0, 0.01]
	Optimisation completed wih following outcome:
		Number of iterations performed: 14
		Number of expectation evaluations performed: 57
		Success: True
		Optimiser message: Optimization terminated successfully.
	Optimised mixer unitary parameters: 0.358
	Optimised Hamiltonian unitary parameters: 1.16

Current Cut Approximation Ratio: 0.8229297003698184
State Probabilities:
State 000: Probability 0.0044
State 001: Probability 0.1116
State 010: Probability 0.0916
State 011: Probability 0.2924
State 100: Probability 0.2924
State 101: Probability 0.0916
State 110: Probability 0.1116
State 111: Probability 0.0044


Finding Best Mixer for layer 2...
	Best mixer is X2 with gradient magnitude 0.21425298911048063

Optimising layer 2...
	Initial Parameter Guesses: [0.358326085819

In [25]:
for key in adapt_qaoa_solution:
    if key == 'all_mixers':
        continue
    print(key+':', adapt_qaoa_solution[key])

cut_approx_ratios: [0.5416967314063463, 0.8229297003698184, 0.998635495169607]
ham_approx_ratios: [0.0, 0.6136394571796567, 0.9970227032537209]
best_mixers: ['standard_x', 'X2']
Probabilities: [1.39134673e-09 1.01887850e-03 6.42231725e-04 4.98338888e-01
 4.98338888e-01 6.42231725e-04 1.01887850e-03 1.39134671e-09]
best_mixer_parameters: [0.7873136409620807, -0.9175011663034471]
best_ham_parameters: [1.9379397478028648, 0.002023738374564865]


Finally, we solve the problem using the Dynamic ADAPT-QAOA which determines at each layer whether it is beneficial to the classical optimisation to include a Hamiltonian unitary or not. To do this, it is useful to generate a dictionary containing splits of each mixer operator into two operators, one which commutes with the Hamiltonian, and one which anti-commutes with it. This is possible for all mixers which are single Pauli strings.

In [19]:
# pauli_mixers_split_ops_dict = build_operators.split_all_mixers(graph[0])
# filtered_pauli_mixers_split_ops_dict = {key: value for key, value in pauli_mixers_split_ops_dict.items() if 'Y' not in key}


In [20]:
# dynamic_adapt_qaoa_solution = get_data.run_dynamic_adapt_qaoa(graph[0], filtered_pauli_ops_dict, filtered_gradient_ops_dict, filtered_pauli_mixers_split_ops_dict, max_depth=circuit_depth)

In [21]:
# for key in dynamic_adapt_qaoa_solution:
#     if key == 'all_mixers':
#         continue
#     print(key+':', dynamic_adapt_qaoa_solution[key])

Overall, we see that the three algorithm implementations converge to a good approximation ratio, with the adaptive problem-tailored ones achieving better results. The dynamic algorithm converges quicker compared to the non-dynamic version.

In [12]:
import re
import qiskit
import json
import time
import numpy as np

from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.converters import circuit_to_dag

from fast_generator import fc_tree_commute_recur_lookahead_fast
from absorption import extract_CNOT_network, update_probabilities

# from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from utilities import generate_pauli_strings
from circuit_generator import generate_opt_circuit, construct_qcc_circuit

In [37]:
#QAOA for MAXCUT 
# cost_hamiltonian = ['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII']
# mixer_hamiltonian = ['XIIII', 'IXIII', 'IIXII', 'IIIXI', 'IIIIX']
# test_paulis = (cost_hamiltonian + mixer_hamiltonian)* 2
# test_params = [3.271] * 6 +   [2.874] * 5 + [2.641] * 6 + [0.9339]* 5


cost_hamiltonian = build_operators.cut_hamiltonian(graph[0])[1]
mixer_hamiltonian = adapt_qaoa_solution['best_mixers']
weights = graph[1]
cost_parameters = adapt_qaoa_solution['best_ham_parameters']
mixer_parameters = adapt_qaoa_solution['best_mixer_parameters']


# test_paulis = #alternate between cost_hamiltonian and one item from the mixer_hamiltonian list
# test_params = 

test_paulis = []
test_params = []

# Alternate between cost_hamiltonian and mixer_hamiltonian elements
for i in range(len(mixer_hamiltonian)):
    # Add cost Hamiltonian terms
    test_paulis.extend(cost_hamiltonian)
    test_params.extend([-1 * cost_parameters[i] * w for w in weights])  # Multiply cost parameters by -1
    
    # Add corresponding mixer Hamiltonian term
    test_paulis.append(mixer_hamiltonian[i])
    test_params.append(mixer_parameters[i] * 2)  # Multiply mixer parameters by 2

# Print results to verify
print("Pauli Strings:", test_paulis)
print("Coefficients:", test_params)


def expand_pauli_strings(pauli_strings, coefficients):
    expanded_paulis = []
    expanded_coeffs = []
    replacement_terms = {
        'standard_x': ['IIX', 'IXI', 'XII'],  # Replace 'standard_x' with 3 terms
        'X2': ['IIX']  # Replace 'X2' with 'IIX'
    }

    for i, pauli in enumerate(pauli_strings):
        if pauli in replacement_terms:
            expanded_paulis.extend(replacement_terms[pauli])  # Replace with corresponding terms
            expanded_coeffs.extend([coefficients[i]] * len(replacement_terms[pauli]))  # Repeat coefficient
        else:
            expanded_paulis.append(pauli)
            expanded_coeffs.append(coefficients[i])

    return expanded_paulis, expanded_coeffs

start_time = time.time()
opt_qc, append_clifford, sorted_entanglers = fc_tree_commute_recur_lookahead_fast(entanglers=expand_pauli_strings(test_paulis, test_params)[0], params=expand_pauli_strings(test_paulis, test_params)[1], barrier=False)
opt_qiskit = transpile(opt_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])

#need to add a layer of hadamard gates:
for i in range(0, len(test_paulis[0])):
    opt_qc.h(i)
opt_qc.measure_active()
end_time = time.time()
combined_time = end_time - start_time


origin_qc = construct_qcc_circuit(entanglers = test_paulis, params = test_params, barrier=False)
#generate the qiskit optimized circuit
origin_qiskit = transpile(origin_qc, optimization_level = 3, basis_gates = ["cx", "sx", "x", "rz"])
origin_qiskit.measure_active() 
origin_qiskit.count_ops()['cx']

expand_pauli_strings(test_paulis, test_params)[0]

SyntaxError: positional argument follows keyword argument (1777513662.py, line 55)

In [13]:
# import numpy as np
# from scipy.sparse import csr_matrix
# from qiskit.quantum_info import SparsePauliOp

# # Assuming 'sparse_matrix' is your sparse matrix of a pure state vector
# # Convert the sparse matrix to a dense array (assuming it's a column vector)
# state_vector = adapt_qaoa_solution['all_den_mats'].toarray().flatten()  # Flatten if it's a column vector

# # Create the density matrix
# density_matrix = np.outer(state_vector, state_vector.conj())

# # Ensure it is a valid density matrix
# # Check Hermitian property
# # print(np.allclose(density_matrix, density_matrix.conj().T))
# # Check trace
# # print(np.isclose(np.trace(density_matrix), 1.0))

# test_paulis = SparsePauliOp.from_operator(density_matrix).paulis[1:]
# test_params = SparsePauliOp.from_operator(density_matrix).coeffs.real[1:]

In [14]:
# state_vector = dynamic_adapt_qaoa_solution['all_den_mats'].toarray().flatten()  # Flatten if it's a column vector

# # Create the density matrix
# density_matrix = np.outer(state_vector, state_vector.conj())
# print(density_matrix.shape)
# test_paulis = SparsePauliOp.from_operator(density_matrix).paulis[1:]
# test_params = SparsePauliOp.from_operator(density_matrix).coeffs.real[1:]